In [74]:
import json
import numpy as np
import nltk
from pprint import pprint
from nltk.corpus import wordnet
from string import punctuation
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer 
import gensim 
from gensim.models import Word2Vec, KeyedVectors
import os
import re
import pandas as pd

nltk.download("wordnet")
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()


[nltk_data] Downloading package wordnet to /home/vatsal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/vatsal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [75]:
def entry(line): 
    w, c = line.split("\t", 2)
    return (w, int(c))

dict_path = "./dict.txt"
dictionary = dict(entry(line) for line in open(dict_path))
max_word_length = max(map(len, dictionary))
total = float(sum(dictionary.values()))
cleanup = re.compile(r'[^a-z0-9]')

def word_prob(word): 
    return dictionary.get(word, 0) / total

def segment(text): 
    text = re.sub(cleanup, ' ', text)
    probs, lasts = [1.0], [0]
    for i in range(1,len(text) + 1):
        prob_k, k = max((probs[j] * word_prob(text[j:i]), j)for j in range(max(0, i - max_word_length), i))
        probs.append(prob_k)
        lasts.append(k)
    words = []
    i = len(text)
    while i > 0:
        words.append(text[lasts[i]:i])
        i = lasts[i]
    words.reverse()
    return words

In [76]:
from glove import Corpus, Glove
corpus = Corpus() 

In [77]:
top100 = {}
top100['travel'] = ['#travel', '#wanderlust', '#nature', '#travelling', '#traveling', '#traveller', '#photography', '#traveler', '#trip', '#travels', '#vacation', '#love', '#travelers', '#adventure', '#tourist', '#landscape', '#travellers', '#explore', '#holiday', '#beautiful', '#tourism', '#hiking', '#beach', '#photo', '#sunset', '#photographer', '#mountains', '#globetrotter', '#summer', '#art', '#sky', '#treking', '#europe', '#view', '#architecture', '#sea', '#happy', '#fun', '#city', '#sun', '#lifestyle', '#amazing', '#wanderer', '#italy', '#follow', '#backpacking', '#life', '#visiting', '#fashion', '#autumn', '#explorer', '#ocean', '#outdoors', '#india', '#world', '#mountain', '#beauty', '#spain', '#backpacker', '#style', '#like', '#france', '#exploring', '#trekking', '#clouds', '#asia', '#me', '#friends', '#usa', '#canon', '#happiness', '#blogger', '#holidays', '#ig', '#sunrise', '#smile', '#germany', '#girl', '#island', '#wander', '#paradise', '#turkey', '#discover', '#italia', '#voyage', '#flowers', '#landscapes', '#sightseeing', '#outdoor', '#history', '#indonesia', '#cute', '#forest', '#paris', '#food', '#australia', '#bali', '#pic', '#beaches', '#inspiration']
top100['food'] = ['#foodie', '#food', '#delicious', '#yummy', '#foodies', '#foods', '#eat', '#breakfast', '#dinner', '#tasty', '#cooking', '#lunch', '#homemade', '#dessert', '#love', '#eating', '#healthy', '#sweet', '#restaurant', '#photography', '#hungry', '#chef', '#blogger', '#travel', '#chocolate', '#baking', '#follow', '#cake', '#vegan', '#fresh', '#chicken', '#like', '#cook', '#amazing', '#blog', '#happy', '#favorite', '#brunch', '#coffee', '#weekend', '#fit', '#vegetarian', '#pasta', '#beautiful', '#pastry', '#fitness', '#gourmet', '#morning', '#desserts', '#seafood', '#eater', '#lifestyle', '#sweets', '#cafe', '#recipes', '#icecream', '#pizza', '#italy', '#culinary', '#candy', '#cheese', '#photographer', '#photo', '#cakes', '#recipe', '#noodles', '#diet', '#eats', '#rice', '#friends', '#cuisine', '#porridge', '#salad', '#nutrition', '#india', '#indonesia', '#bread', '#mornings', '#drinks', '#art', '#life', '#fun', '#gastronomy', '#sunday', '#cookies', '#kitchen', '#gym', '#me', '#beef', '#sushi', '#cupcakes', '#bake', '#spicy', '#saturday', '#fish', '#catering', '#burger', '#snack', '#music', '#delhi']
top100['baby'] = ['#baby', '#kids', '#love', '#babies', '#cute', '#family', '#mom', '#fashion', '#newborn', '#children', '#happy', '#beautiful', '#photography', '#girl', '#babys', '#adorable', '#sweet', '#motherhood', '#pregnancy', '#handmade', '#funny', '#lovely', '#smile', '#mommy', '#life', '#daughter', '#style', '#follow', '#sweetheart', '#like', '#pregnant', '#beauty', '#precious', '#kid', '#child', '#fun', '#girls', '#mummy', '#tiny', '#twins', '#amazing', '#toddler', '#model', '#photo', '#mama', '#toys', '#boy', '#sleep', '#matching', '#autumn', '#enjoy', '#friends', '#pretty', '#art', '#parenting', '#live', '#mother', '#me', '#amor', '#photographer', '#princess', '#fairy', '#sale', '#happiness', '#boys', '#boutique', '#lifestyle', '#portrait', '#fall', '#son', '#pink', '#nature', '#childhood', '#cuteness', '#maternity', '#cool', '#travel', '#angel', '#best', '#halloween', '#play', '#weekend', '#october', '#dad', '#innocent', '#brand', '#makeup', '#home', '#parenthood', '#nice', '#little', '#sunday', '#flowers', '#mum', '#canon', '#infant', '#party', '#wedding', '#summer', '#hot']
top100['jewellery'] = ['#jewellery', '#jewelry', '#fashion', '#accessories', '#earrings', '#necklace', '#style', '#jewels', '#handmade', '#beautiful', '#gemstone', '#trendy', '#gold', '#jewel', '#bracelet', '#rings', '#love', '#gems', '#silver', '#bracelets', '#design', '#ring', '#gemstones', '#crystals', '#shopping', '#stylish', '#art', '#pendant', '#diamond', '#cute', '#luxury', '#necklaces', '#bling', '#india', '#watches', '#handcrafted', '#designer', '#bijoux', '#trending', '#earring', '#bangles', '#bride', '#wedding', '#beauty', '#stone', '#tourmaline', '#charm', '#traditional', '#hyderabad', '#wholesaler', '#diamonds', '#bridal', '#photography', '#accessory', '#cabochons', '#tourmalines', '#stones', '#gem', '#goldplated', '#semiprecious', '#mumbai', '#silversmith', '#pendents', '#k', '#indian', '#sapphire', '#chennai', '#jewellers', '#girls', '#sale', '#jeweller', '#delhi', '#follow', '#fashionable', '#gift', '#bangalore', '#women', '#london', '#unique', '#ethnic', '#rawalpindi', '#crystal', '#vintage', '#oxidised', '#emerald', '#dubai', '#tucson', '#peridot', '#beads', '#makeup', '#saree', '#kunzite', '#chic', '#firecracker', '#pearl', '#lahore', '#choker', '#islamabad', '#wholesale', '#combo']
# top100['selfie'] = ['#me', '#love', '#follow', '#smile', '#photography', '#style', '#beautiful', '#fashion', '#happy', '#cute', '#fun', '#girl', '#life', '#portrait', '#like', '#pretty', '#photo', '#handsome', '#travel', '#nature', '#eyes', '#face', '#hair', '#summer', '#art', '#friends', '#amazing', '#model', '#food', '#beauty', '#pic', '#daily', '#girls', '#lifestyle', '#look', '#makeup', '#swag', '#photographer', '#fitness', '#autumn', '#cool', '#family', '#sky', '#picture', '#likes', '#sun', '#mood', '#music', '#followers', '#outfit', '#boy', '#colorful', '#self', '#motivation', '#lovely', '#gay', '#pose', '#landscape', '#black', '#inspiration', '#happiness', '#halloween', '#artist', '#gym', '#liker', '#pink', '#portraits', '#blogger', '#fit', '#sunset', '#wanderlust', '#design', '#funny', '#italy', '#traveling', '#hot', '#awesome', '#holiday', '#following', '#photos', '#india', '#night', '#blue', '#weekend', '#view', '#october', '#party', '#trip', '#red', '#tattoo', '#man', '#looks', '#stylish', '#sea', '#nice', '#good', '#beard', '#beach', '#woman', '#aesthetic']
top100['pet'] = ['#pet', '#pets', '#dog', '#cute', '#animals', '#love', '#dogs', '#cat', '#puppy', '#animal', '#cats', '#kitten', '#nature', '#kitty', '#meow', '#adorable', '#puppies', '#kittens', '#doggy', '#doggo', '#pup', '#photography', '#happy', '#funny', '#hound', '#eyes', '#beautiful', '#bunny', '#poodle', '#chihuahua', '#follow', '#rabbit', '#fluffy', '#paws', '#family', '#photo', '#lovely', '#chat', '#furry', '#daily', '#pomeranian', '#labrador', '#fun', '#autumn', '#sweet', '#maltese', '#smile', '#life', '#art', '#bunnies', '#hamster', '#baby', '#bird', '#me', '#world', '#bulldog', '#like', '#sleeping', '#tuesday', '#summer', '#parrots', '#husky', '#amor', '#rabbits', '#tot', '#zoo', '#doggie', '#meme', '#pug', '#istanbul', '#wildlife', '#beauty', '#adopt', '#girl', '#meowed', '#birds', '#purr', '#rescue', '#cavy', '#japan', '#doggies', '#halloween', '#photographer', '#woof', '#playtime', '#nice', '#portrait', '#relax', '#parakeet', '#singapore', '#parrot', '#fashion', '#fall', '#chihuahuas', '#friends', '#travel', '#morning', '#wolf', '#sweden', '#monday']
top100['art'] = ['#art', '#artist', '#drawing', '#artwork', '#illustration', '#sketch', '#painting', '#draw', '#sketchbook', '#creative', '#design', '#love', '#photography', '#beautiful', '#pencil', '#ink', '#illustrator', '#drawings', '#portrait', '#arts', '#gallery', '#artistic', '#fashion', '#paint', '#doodle', '#nature', '#picture', '#watercolor', '#graphic', '#color', '#photo', '#sketching', '#music', '#style', '#pen', '#anime', '#cute', '#paintings', '#follow', '#paper', '#graphics', '#artists', '#like', '#travel', '#abstract', '#tattoo', '#cartoon', '#girl', '#masterpiece', '#sketches', '#beauty', '#inspiration', '#procreate', '#designer', '#comics', '#artworks', '#photographer', '#life', '#digital', '#illustrations', '#model', '#handmade', '#watercolour', '#draws', '#happy', '#painter', '#black', '#colorful', '#colors', '#comic', '#fun', '#landscape', '#exhibition', '#cool', '#colour', '#architecture', '#aesthetic', '#acrylic', '#smile', '#graffiti', '#arty', '#tattoos', '#amazing', '#lifestyle', '#sculpture', '#blue', '#rap', '#halloween', '#inked', '#funny', '#fantasy', '#character', '#me', '#summer', '#singer', '#studio', '#logo', '#collage', '#doodles', '#pictures']
top100['architecture'] = ['#architecture', '#design', '#art', '#photography', '#building', '#travel', '#architect', '#city', '#buildings', '#urban', '#beautiful', '#style', '#minimal', '#street', '#architectural', '#interior', '#skyscraper', '#perspective', '#abstract', '#lines', '#town', '#architectures', '#geometric', '#designer', '#composition', '#geometry', '#arts', '#cities', '#home', '#landscape', '#nature', '#photo', '#photographer', '#interiors', '#house', '#italy', '#love', '#cityscape', '#modern', '#decor', '#pattern', '#luxury', '#sky', '#construction', '#architects', '#inspiration', '#arch', '#traveling', '#minimalism', '#render', '#wanderlust', '#sketch', '#rendering', '#europe', '#lifestyle', '#history', '#autumn', '#view', '#decoration', '#light', '#trip', '#amazing', '#concrete', '#paris', '#sunset', '#facade', '#artist', '#modernism', '#contemporary', '#sun', '#london', '#travelling', '#furniture', '#ig', '#life', '#france', '#church', '#traveller', '#canon', '#pic', '#explore', '#picture', '#details', '#colors', '#traveler', '#vacation', '#tourism', '#italia', '#milano', '#clouds', '#engineering', '#fashion', '#germany', '#spain', '#villa', '#wood', '#3d', '#exterior', '#blue', '#artwork']
top100['nature'] = ['#nature', '#photography', '#landscape', '#travel', '#beautiful', '#love', '#autumn', '#photo', '#sky', '#photographer', '#mountains', '#sunset', '#outdoors', '#clouds', '#art', '#flowers', '#beauty', '#tree', '#wanderlust', '#follow', '#amazing', '#adventure', '#trees', '#forest', '#view', '#explore', '#green', '#summer', '#life', '#sun', '#hiking', '#colors', '#wildlife', '#mountain', '#travelling', '#fashion', '#like', '#canon', '#trip', '#sea', '#ig', '#happy', '#water', '#natural', '#fall', '#outdoor', '#beach', '#style', '#traveling', '#fun', '#blue', '#landscapes', '#sunrise', '#flower', '#animals', '#lake', '#cute', '#picture', '#macro', '#me', '#river', '#earth', '#natures', '#friends', '#leaves', '#followers', '#vacation', '#plants', '#weather', '#smile', '#garden', '#india', '#girl', '#pic', '#november', '#photos', '#world', '#wild', '#birds', '#model', '#traveller', '#peace', '#lifestyle', '#bird', '#walk', '#animal', '#holiday', '#discover', '#trekking', '#portrait', '#morning', '#nice', '#sunday', '#ocean', '#sunsets', '#traveler', '#fitness', '#rain', '#italy', '#winter']

for topic in top100.keys():
    top100[topic] = [r.replace('#', '') for r in top100[topic]]

In [78]:
glob_corpora = {}

for topic in top100.keys():
    glob_corpora[topic] = []

In [79]:
top100_mapping = {}
for key in top100.keys():
    top100_stemmed = [ps.stem(word) for word in top100[key]]
    top100_mapping[key]={}
    for i in range(0,100):
        top100_mapping[key][top100_stemmed[i]]=top100[key][i]

In [80]:
def strip_punc(s):
    new_str = ""
    for c in s:
        if c in punctuation:
            new_str += " "
        else:
            new_str += c
    return new_str

In [89]:
for topic in top100.keys():
    for r, d, f in os.walk("./jsons/"+topic):
        print(topic)
        df = pd.read_csv('./csv/train/final_train_'+topic+".csv")
        dic = dict()
        for file in f:
            path = r+"/"+file
            print(path)
            if '.json' in file:
                file_ptr = open(path, "r")
                tmp_dic = json.load(file_ptr)
                dic.update(tmp_dic)
                
        print(len(dic))
        for index,row in df.iterrows():
            p_id = row['post_id'].split('.')[0]
            try:
                text = strip_punc(dic[p_id]['text_des'].lower())
            except:
                continue
            toks = word_tokenize(text)
            toks_ = []
            for tok in toks:
                tok = segment(tok)
                for t in tok:
                    if not wordnet.synsets(t) or len(t) < 3:
                        continue

                    t = ps.stem(t)
                    if t not in stop_words:
                        toks_.append(t)

            if toks_:
                for topic in top100.keys():
                    if topic in path:
                        glob_corpora[topic].append(toks_)

architecture
./jsons/architecture/modernarchitecture.json
./jsons/architecture/architecture.json
./jsons/architecture/architecturelovers.json
./jsons/architecture/architecturephotography.json
./jsons/architecture/architecture_hunter.json
./jsons/architecture/architectureporn.json
5331



KeyboardInterrupt



In [82]:
# for r, d, f in os.walk("./jsons"):
#     for file in f:
#         path = r+"/"+file
#         print(path)
#         if '.json' in file:
#             file_ptr = open(path, "r")
#             dic = json.load(file_ptr)
#             for post in dic:
#                 text = strip_punc(dic[post]['text_des'].lower())
#                 toks = word_tokenize(text)
#                 toks_ = []
#                 for tok in toks:
#                     tok = segment(tok)
#                     for t in tok:
#                         if not wordnet.synsets(t) or len(t) < 3:
#                             continue

#                         t = ps.stem(t)
#                         if t not in stop_words:
#                             toks_.append(t)

#                 if toks_:
#                     for topic in top100.keys():
#                         if topic in path:
#                             glob_corpora[topic].append(toks_)

In [83]:
print(glob_corpora.keys())
for topic in top100.keys():
    print(topic,len(glob_corpora[topic]))

dict_keys(['architecture', 'nature'])
architecture 4507
nature 5020


In [84]:
# Using word2vec for training
glob_model = {}

model = KeyedVectors.load_word2vec_format("./glove.6B/glove.6B.300d.w2vformat.txt", binary=False)

In [85]:
for topic in top100.keys():
    print(topic)
    glob_model[topic] = Word2Vec(glob_corpora[topic], size = 300, min_count = 2)
#     glob_model[topic].build_vocab(glob_corpora[topic])
    total_examples = glob_model[topic].corpus_count
    glob_model[topic].build_vocab([list(model.vocab.keys())], update=True)
    glob_model[topic].intersect_word2vec_format("./glove.6B/glove.6B.300d.w2vformat.txt", binary=False, lockf=1.0)
    glob_model[topic].train(glob_corpora[topic], total_examples=total_examples, epochs= glob_model[topic].iter)
    glob_model[topic].save("./models/word2vec."+topic)

architecture


/home/vatsal/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  


nature


/home/vatsal/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  


In [54]:
# model = gensim.models.Word2Vec.load("./models/modelName.model")

In [55]:
# print(glob_model.wv.most_similar(positive = 'grass', topn = 20))

In [86]:
# Using Glove for training
glove = {}

for topic in top100.keys():
    corpus.fit(glob_corpora[topic], window = 10)
    glove[topic] = Glove(no_components = 100, learning_rate = 0.05)
    glove[topic].fit(corpus.matrix, epochs = 30, no_threads = 4, verbose = True)
    glove[topic].add_dictionary(corpus.dictionary)
    glove[topic].save('./models/glove.model.'+topic)
    glove[topic].add_dictionary(corpus.dictionary)

Performing 30 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Performing 30 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29


In [87]:
print(top100.keys())

dict_keys(['architecture', 'nature'])


In [58]:
# Preprocessing of input text

# input_sentence = "#forest#green#wanderlust#tour"
# input_sentence = "Chocolate Entremet by Chef @karim.bourgi @beechefbyhansovando @hansovando @academia_fouet 🔥🔥🔥 #cakes #yummy #instagood #fan #tutorial #colourful #foodgasm #instacake #chocolate #chocolates #delicious #yummyyummy #food #cakestagram #amazing #instagram #pic #instayum #partage #socute #instagood #insta #instafood #follower #baking #instasweet #chocolate #foodstagram #pastrychef #instafood #instagram #pic #thafoodheaven #cakestagram #instayum #cakepops"
input_sentence = "Straight out of The Sound of Music scenery is what you can wake up to, hiking and mountain biking your days away and enjoying Tyrolean hospitality.⁠"
text = strip_punc(input_sentence.lower())
toks = word_tokenize(text)
toks_ = []
for tok in toks:
    tok = segment(tok)
    for t in tok:
        if wordnet.synsets(t):
            t = ps.stem(t)
            if t not in stop_words and len(t)>2:
                toks_.append(t)

input_words = toks_
print(input_words)

topic = "travel"
# topic = "food"

['straight', 'sound', 'music', 'sceneri', 'wake', 'hike', 'mountain', 'bike', 'day', 'away', 'enjoy', 'role', 'hospit']


In [59]:
# Calculating sentence embedding using word2vec

sent = []
for word in input_words:
    if word in glob_model[topic].wv.vocab:
        sent.append(glob_model[topic].wv[word])
    
print(len(sent))
sent = np.array(sent)
print(sent.shape)
embed = np.average(sent, axis = 0)
print(embed.shape)

KeyError: 'travel'

In [ ]:
# Finding similarity of top 100 hashtags using word2vec

ans = []
for hasht in top100_mapping[topic]:
    try:
        v1 = glob_model[topic].wv[hasht]
        simi = np.dot(v1, embed) / (np.linalg.norm(v1) * np.linalg.norm(embed))
        ans.append((simi, top100_mapping[topic][hasht]))
    except:
        pass

ans = sorted(ans, reverse = True)
print(ans[:20])

In [ ]:
# Finally top k for word2vec

k = 10
topk_word2vec = [h[1] for h in ans[:k]]
print(topk_word2vec)

In [ ]:
# Calculating sentence embedding using glove

sent = []
print(input_words)
for word in input_words:
    if word in glove[topic].dictionary:
        sent.append(glove[topic].word_vectors[glove[topic].dictionary[word]])
    
sent = np.array(sent)
print(sent.shape)
embed = np.average(sent, axis = 0)
print(embed.shape)

In [ ]:
print(glove.keys())

In [ ]:
# Finding similarity using glove embeddings

ans2 = []
for hasht in top100_mapping[topic]:
    try:
        v1 = glove[topic].word_vectors[glove[topic].dictionary[hasht]]
        simi = np.dot(v1, embed) / (np.linalg.norm(v1) * np.linalg.norm(embed))
        ans2.append((simi, top100_mapping[topic][hasht]))
    except:
        pass

ans2 = sorted(ans2, reverse = True)
print(ans2[:20])

In [ ]:
# Finally topk for glove
k = 10

topk_glove = [h[1] for h in ans2[:k]]
print(topk_glove)

In [ ]:
print("Glove output",topk_glove)
print("Word2vec output", topk_word2vec)